In [1]:
from pathlib import Path
import os
cwd_path = Path.cwd(); set_path = str(cwd_path.parent); os.chdir(set_path)

In [2]:
from TargetDistributions.BayesianNN import PosteriorBNN
from FittedModels.Models.FlowModel import FlowModel
from FittedModels.experimental.train_AIS import AIS_trainer
from FittedModels.utils import plot_history
import matplotlib.pyplot as plt
import torch

### Expectation Function

In [3]:
def expectation_function(x):
    A = torch.ones((x.shape[-1], x.shape[-1]))
    return torch.einsum("bi,ij,bj->b", x, A, x)

# Specifiy target distribution and key param

In [4]:
epochs = 5
n_samples_estimation = int(1e4)
target = PosteriorBNN(n_datapoints=10, x_dim=2, y_dim=2, n_hidden_layers=2, layer_width=5)
dim = target.n_parameters

# DReG + AIS

In [5]:
learnt_sampler = FlowModel(x_dim=dim) #, flow_type="RealNVP")
tester = AIS_trainer(target, learnt_sampler, loss_type="DReG", n_distributions=5, n_updates_Metropolis=5)
history = tester.train(epochs, batch_size=100)

  0%|                                                                                                                                                            | 0/5 [00:05<?, ?it/s]


ValueError: The parameter scale has invalid values

In [ ]:
plot_history(history)

# DReG without AIS

In [ ]:
from FittedModels.train import LearntDistributionManager

# KL without AIS

In [ ]:
from ImportanceSampling.VanillaImportanceSampler import VanillaImportanceSampling

In [ ]:
learnt_sampler = FlowModel(x_dim=dim)
tester = LearntDistributionManager(target, learnt_sampler, VanillaImportanceSampling, loss_type="kl")
history = tester.train(epochs, batch_size=200)

In [ ]:
plot_history(history)
plt.show()

In [ ]:
expectation, info = tester.estimate_expectation(n_samples_estimation, expectation_function)
print(f"effective sample size is {info['effective_sample_size']} \n"
     f"variance in weights is {torch.var(info['normalised_sampling_weights'])}")